# Testing and test-driven development

Testing is an important part of software engineering and scientific computing. There's a saying:

> Untested code is broken code.

Which is to say that, if you are not writing tests for your code, you should assume it's broken. 

Watch this video https://www.youtube.com/watch?v=FxSsnHeWQBY

In [11]:
import numpy as np

## Simple test

In test-driven development, we go so far as to write the test, which fails of course, and then writing the function that passes the test:

In [102]:
def test_add():
    """
    Test the add() function. Which I haven't defined yet.
    """
    result = add(2, 3)
    answer = 5
    assert result == answer
    
    result = add("2", "3")
    assert result == answer
    
def add(a, b):
    """
    Add two numbers.
    """
    return float(a) + float(b)

test_add()

No result means the test passed!

Here's a more involved example:

In [35]:
def test_shuey():
    vp1, vs1, rho1 = 3000, 1414, 2400
    vp2, vs2, rho2 = 3100, 1500, 2420
    theta = 0
    result = shuey(vp1, vs1, rho1, vp2, vs2, rho2, theta)
    answer = 0.020543
    assert abs(result - answer) < 0.00001
    
    #!--
    theta = 20
    result = shuey(vp1, vs1, rho1, vp2, vs2, rho2, theta)
    answer = 0.015715
    assert abs(result - answer) < 0.00001
    #--!
    
    return

In [36]:
def shuey(vp1, vs1, rho1, vp2, vs2, rho2, theta):
    theta = np.radians(theta)

    drho = rho2 - rho1
    dvp = vp2 - vp1
    dvs = vs2 - vs1
    rho = (rho1 + rho2) / 2
    vp = (vp1 + vp2) / 2
    vs = (vs1 + vs2) / 2

    r0 = 0.5 * (dvp/vp + drho/rho)
    g = 0.5 * dvp/vp - 2 * (vs**2/vp**2) * (drho/rho + 2 * dvs/vs)

    return r0 + g * np.sin(theta)**2

In [37]:
test_shuey()

Again, no result means the test passed!

Why the weird assert, why no just `assert actual == answer`? Float imprecision!

In [13]:
0.1 + 0.2

0.30000000000000004

There is an alternative in `np.testing` (and many testing frameworks will have ways to make testing approximate results easier):

In [41]:
vp1, vs1, rho1 = 3000, 1414, 2400
vp2, vs2, rho2 = 3100, 1500, 2420
theta = 0
answer = 0.020543
np.testing.assert_almost_equal(shuey(vp1, vs1, rho1, vp2, vs2, rho2, theta), answer, decimal=5)

## Exercise

Add a test for another angle to the test function. Use the fact that the correct result for an angle of 20 degrees is 0.015714797299808014. Make sure the test passes when you run the following code block:

In [ ]:
test_shuey()

## `py.test`

`py.test` is a testing framework. There are others:

- `unittest` — the one that comes with Python.
- `nose` — seems popular; I've never used it.

`py.test` lets you write fairly striaghtforward code for testing (compared to `unittest`, which I find has a lot of boilerplate). And it has a nice system of plugins for handy things like testing your `matplotlib` plots, and getting 'coverage' reports.

We are not going to go into `py.test` now; the example project `project` implements it.

## `doctest`

Make your docstrings work for a living!

In [68]:
def quad(x, a=1, b=1, c=0):
    """
    Returns the quadratic function of x,
    a.x^2 + b.x + c
    where
    a = b = 1 and c = 0.
    
    Examples:
    >>> quad(10)
    110
    >>> quad(10, a=3, b=2, c=1)
    321
    """
    return a*x**2 + b*x + c

In [69]:
quad(10, a=3, b=2, c=1)

321

We can run the doctests in a single function like so:

In [70]:
import doctest
doctest.run_docstring_examples(quad, globals(), verbose=True)

Finding tests in NoName
Trying:
    quad(10)
Expecting:
    110
ok
Trying:
    quad(10, a=3, b=2, c=1)
Expecting:
    321
ok


We can test everything in the current 'module' (in our case, the Notebook) like so:

In [ ]:
doctest.testmod()

## Exercise

Write doctests for our `add()` function:

In [ ]:
def add(a, b):
    """
    Add two numbers.
    """
    return float(a) + float(b)

And make sure your test passes:

In [ ]:
doctest.run_docstring_examples(add, globals(), verbose=True)

## Notebook magic

In IPython (or the Notebook) we can even define a magic to make this sort of thing easier:

In [71]:
from IPython.core.magic import register_line_magic

@register_line_magic
def testit(_):
    return doctest.testmod()

In [72]:
%testit

TestResults(failed=0, attempted=2)

In [79]:
@register_line_magic
def testfunc(func):
    """
    Using eval() might not be a great idea, but we
    are only passing it to doctest so maybe it's OK.
    """
    if not func:
        raise SyntaxError("You must provide a function to test.")
    return doctest.run_docstring_examples(eval(func), globals(), verbose=True)

In [81]:
%testfunc quad

Finding tests in NoName
Trying:
    quad(10)
Expecting:
    110
ok
Trying:
    quad(10, a=3, b=2, c=1)
Expecting:
    321
ok


## ...or with a decorator

Or we could define a decorator to test functions:

In [97]:
from functools import wraps
import doctest

def test(func):
    @wraps(func)
    def f(*args, **kwargs):
        return func(*args, **kwargs)
    doctest.run_docstring_examples(func, globals())
    return f

I'll pass this with a failing test so we can see that something is happening.

In [101]:
@test
def quad(x, a=1, b=1, c=0):
    """
    Returns the quadratic function of x,
    a.x^2 + b.x + c
    where
    a = b = 1 and c = 0.
    
    Examples:
    >>> quad(10)
    110
    >>> quad(10, a=3, b=2, c=1)
    3210
    """
    return a*x**2 + b*x + c

**********************************************************************
File "__main__", line 12, in NoName
Failed example:
    quad(10, a=3, b=2, c=1)
Expected:
    3210
Got:
    321


## Other testing topics

### Coverage

### Mocks

### Web and UI testing

### Continuous integration
